## Run Baseline ResNet-50 

TASK: Detect Insect Type \
Dataset: iNaturalist \
Model: Resnet-50 (pretrained)

In [2]:
# pip install datasets
# pip install scikit-learn

In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, KFold
import os
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

/opt/anaconda3/envs/resnet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data Cleaning

In [4]:
# https://huggingface.co/datasets/sxj1215/inaturalist
dataset = load_dataset("sxj1215/inaturalist", split="train")

In [5]:
# using Matilda's label mapping (take from utils/label_mapping.py in the future)
iNat_to_clean_map = {
    # Ants
    "Camponotus planatus": "Ant",
    
    # Bees
    "Agapostemon virescens": "Bee",

    # Beetles
    "Alaus lusciosus": "Beetle",
    "Buprestis aurulenta": "Beetle",
    "Chrysolina americana": "Beetle",
    "Cicindela hirticollis": "Beetle",
    "Epicauta pennsylvanica": "Beetle",
    "Polydrusus formosus": "Beetle",
    "Trypoxylus dichotomus": "Beetle",

    # Butterflies
    "Aglais urticae": "Butterfly",
    "Anartia amathea": "Butterfly",
    "Cigaritis lohita": "Butterfly",
    "Euphyes vestris": "Butterfly",
    "Icaricia acmon": "Butterfly",
    "Icaricia lupini": "Butterfly",
    "Panoquina ocola": "Butterfly",

    # # Moths 
    # "Agrotis segetum": "Butterfly",
    # "Aphomia sociella": "Butterfly",
    # "Apogeshna stenialis": "Butterfly",
    # "Cabera pusaria": "Butterfly",
    # "Ceratomia catalpae": "Butterfly",
    # "Enyo lugubris": "Butterfly",
    # "Gastrina cristaria": "Butterfly",
    # "Gymnandrosoma punctidiscanum": "Butterfly",
    # "Heterophleps triguttaria": "Butterfly",
    # "Ochropleura plecta": "Butterfly",
    # "Thaumetopoea processionea": "Butterfly",
    # "Triphosa haesitata": "Butterfly",

    # Dragonflies
    
    # Flies

    # Grasshoppers
    "Gomphocerippus rufus": "Grasshopper",

    # Ladybugs
    "Hippodamia variegata": "Ladybug",

    # Mosquito

    # Spiders
    "Aphonopelma chalcodes": "Spider",
    "Platycryptus undatus": "Spider",

    # Wasps
    "Chlorion aerarium": "Wasp" #this wasp is blue though
}


In [6]:
os.makedirs("images", exist_ok=True)

rows = []
img_id = 0

for row in dataset:
    original_label = row["messages"][1]["content"]

    if original_label not in iNat_to_clean_map:
        continue  # skip unmapped species (?), should there be a different OTHER class

    clean_label = iNat_to_clean_map.get(original_label, original_label)

    image_path = f"images/{img_id}.jpg"
    row["images"][0].save(image_path)

    rows.append(f"{image_path},{clean_label}")
    img_id += 1

OSError: image file is truncated (122 bytes not processed)

In [7]:
with open("labels.csv", "w") as f:
    f.write("image,label\n")
    f.write("\n".join(rows))

Custom Dataset Transformations

In [8]:
# uses RandomResizedCrop, RandomHorizontalFlip to help model generalize
# applies standard ImageNet normalization

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [9]:
# custom PyTorch Dataset class to load images and convert labels to integers

class JPGDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data = data_df  # now accepts a df directly
        self.transform = transform

        # map label strings to integers
        self.classes = sorted(self.data.label.unique())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image = Image.open(row.image).convert("RGB")
        label = self.class_to_idx[row.label]
        if self.transform:
            image = self.transform(image)
        return image, label


Model Architecture/Training (random sample, cross-val)

In [11]:
def train_one_epoch(model, loader):
    model.train()
    correct, total = 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return correct / total

In [12]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
df = pd.read_csv("labels.csv")
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    print(f"\n--- Starting Fold {fold + 1}/5 ---")
    
    # create df for eacg fold
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]
    
    # initialize datasets and loaders
    train_dataset = JPGDataset(train_df, transform=train_transforms)
    val_dataset   = JPGDataset(val_df, transform=val_transforms)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    num_classes = len(train_dataset.classes)

    # re-initialize the model for every fold to prevent weight leakage
    model = models.resnet50(weights="IMAGENET1K_V1")
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    # freeze backbone weights and unfreeze the new head
    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

    # train for 2 epochs per fold
    for epoch in range(2):
        train_acc = train_one_epoch(model, train_loader)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: train={train_acc:.3f}, val={val_acc:.3f}")
    
    fold_accuracies.append(val_acc)

print(f"\nAverage 5-Fold Accuracy: {np.mean(all_fold_results):.3f}")


--- Starting Fold 1/5 ---
Epoch 1: train=0.754, val=0.944


In [ ]:
# save the final weights and class names
torch.save({
    "model": model.state_dict(),
    "classes": train_dataset.classes
}, "resnet50_insects_cv.pth")

Model Architecture/Training (random sample, no cross-val)

In [ ]:
df = pd.read_csv("labels.csv")

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df.label,
    random_state=42
)

train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)


In [ ]:
train_dataset = JPGDataset("train.csv", transform=train_transforms)
val_dataset   = JPGDataset("val.csv", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the pre-trained resnet50 model
model = models.resnet50(weights="IMAGENET1K_V1")

# replace the final layer with new layer with the number of insect classes
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [ ]:
# the backbone weights of the ResNet are frozen
for param in model.parameters():
    param.requires_grad = False

# only the new final layer will be updated during the training (fine-tuning step 1)
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
# model trained using Adam optimizer and CrossEntropyLoss

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.fc.parameters(),
    lr=1e-3
)

In [16]:
epochs = 2

for epoch in range(epochs):
    train_acc = train_one_epoch(model, train_loader)
    val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}: train={train_acc:.3f}, val={val_acc:.3f}")


Epoch 1: train=0.767, val=0.927
Epoch 2: train=0.891, val=0.956


In [ ]:
# save the final weights and class names

torch.save({
    "model": model.state_dict(),
    "classes": train_dataset.classes
}, "resnet50_insects.pth")
